# 使い方
1. 「ファイル」タブに"{name}_data.csv"を入れる
2.  ランタイム→すべてのセルを実行 をクリック
3.  output_zipフォルダ内に入ってる"outut_{current_datetime}.zip"ファイルをダウンロード
4.  ダウンロードしたzipファイルを適当なところに展開
5.  展開したフォルダのなかのcsvファイルがいっぱい入ってるところに同梱の"alpha.m"を入れる
6.  octave上で"alpha.m"を実行
7.  出力された"alpha.csv"に全部のα波が入ってる

### なんかおかしいなってときは
下にあるHOSEI変数を正の整数の範囲でいじってみて。

実行後に下の方に色々でるからそこが

番号 1.5 くらい

動画 4~5 くらい

解答 6.5 くらい

になっていたら値は正しく取れてる(はず)。

### 一番最初に行う処理


In [ ]:
# import類
import sys
import pandas as pd
import os
from datetime import datetime
import shutil
import numpy as np
from scipy.fft import fft

HOSEI = 1  # <==================================== コレ

### 関数定義

In [ ]:
### zip化するための関数

def compress_directory_to_zip(directory_path, zip_output_directory = './output_ZIP/'):
    # 出力ファイル名に日時を組み込む
    current_datetime = datetime.now().strftime("%Y%m%d%H%M%S")
    zip_filename = f'{os.path.basename(os.path.dirname(directory_path))}_{current_datetime}.zip'

    # ZIPファイルのパス
    zip_output_path = os.path.join(zip_output_directory, zip_filename)

    # 指定されたディレクトリをZIPファイルに圧縮
    shutil.make_archive(zip_output_path[:-4], 'zip', directory_path)

    print(f"{os.path.basename(directory_path)}ディレクトリがZIPファイルに圧縮されました: {zip_output_path}")

In [ ]:
### データを分割するための関数
def separate_data(csv_file_path, output_directory='./output/', correction_value=1):
    # 出力する数
    output_num = 200
    #サンプリング周波数 (Hz)
    sampling_frequency = 500

    # 分割されたデータを保存する
    sepalate_data = [[ [], [], [],[],[],[],[] ]]

    # トリガー
    trigger = False
    start = False
    gap = 0
    NT = False
    threshold = 100

    # 入れる場所管理用
    cnt = 0
    num = 0

    # CSVファイルをpandasのデータフレームに読み込む
    df = pd.read_csv(csv_file_path)

    # データフレームをNumPy配列に変換
    data_array = df.values
    print(int (len(data_array) / sampling_frequency))
    end_point = int (len(data_array) / sampling_frequency) -1
    print(end_point)

    for data in data_array:
        if data[4] <= threshold:
            NT = True

        if data[4] > threshold and not trigger:#開始
            trigger = True
            start = True


        elif data[4] > threshold and trigger and NT:#終了
            trigger = False
            NT = False

        if  abs(data[0] - end_point) < 1e-6: #終了で、最後はトリガーがないから、消えるの対策
            trigger = False
            NT = False

        #これは、事象関連電位の特別if文
        #if data[4] == 570.755:
            #trigger = False
            #NT = False


        #if gap < correction_value and trigger:
            #trigger = False
            #start = False
            #gap += 1
            #NT = False

        if trigger:
            sepalate_data[cnt][num].append([data[0], data[1], data[2], data[5], data[6], data[4],data[7]])
        elif start:
            num += 1
            if num > 1:
                sepalate_data.append([ [], [], [],[],[],[],[]])
                cnt += 1
                num = 1
                print(data[0])
                dff = pd.DataFrame(sepalate_data[cnt-1][1], columns=['Time', 'C3', 'C4','Fz', 'Cz','threshold','Pz'])
                csv_output_path = f'{output_directory}outputdata{cnt-1}.csv'
                dff.to_csv(csv_output_path, index=False)

        if cnt > output_num:
            break
    print(cnt)
    print("#########################################")
    for i in range(cnt):
        print("##########  ", i, "  ###########")
        print("番号", len(sepalate_data[i][0])/1000)
        print("動画", len(sepalate_data[i][1])/1000)
        print("解答", len(sepalate_data[i][2])/1000)


In [ ]:
### データを分割するための関数
def separate_data_2(csv_file_path, output_directory='./output/', correction_value=1):
    # 出力する数
    output_num = 120

    # 分割されたデータを保存する
    sepalate_data = [[ [], [], [],[],[],[],[] ]]
    sepalate_data_cnt = []
    sepalate_data_end_cnt = []

    # トリガー
    trigger = False
    start = False
    gap = 0
    NT = False
    threshold = 100

    # 入れる場所管理用
    cnt = 0
    num = 0

    # CSVファイルをpandasのデータフレームに読み込む
    df = pd.read_csv(csv_file_path)

    # データフレームをNumPy配列に変換
    data_array = df.values

#トリガー確認
    for data in data_array:
        if data[4] <= threshold:
            NT = True

        if data[4] > threshold and trigger == False:#開始
            trigger = True
            start = True
            NT = False
            sepalate_data_cnt.append(data[0])


        elif data[4] > threshold and trigger and NT:#終了
            trigger = False
            NT = False
            sepalate_data_end_cnt.append(data[0])
        if cnt > output_num:
            break

    #print(len(sepalate_data_cnt))
    #print(sepalate_data_end_cnt)

    # トリガー初期化
    trigger = False
    start = False
    NT = False
    output_num = len(sepalate_data_cnt) -1
    # 入れる場所管理用初期化
    cnt = 0
    num = 0

#ファイル出力
    loop = 0
    for data in data_array:

        if abs(data[0] - (sepalate_data_cnt[loop] - 3.0)) < 1e-6:#開始
            print(data[0])
            trigger = True
            start = True

        if data[0] == sepalate_data_cnt[loop]:#終了
            trigger = False
            NT = False
            loop += 1

        if trigger:
            sepalate_data[cnt][num].append([data[0], data[1], data[2], data[5], data[6], data[4],data[7]])

        if start and trigger == False:
            sepalate_data.append([ [], [], [],[],[],[],[]])
            cnt += 1
            num = 0

            dff = pd.DataFrame(sepalate_data[cnt-1][0], columns=['Time', 'C3', 'C4','Fz', 'Cz','threshold','Pz'])
            csv_output_path = f'{output_directory}outputdata{cnt-1}.csv'
            dff.to_csv(csv_output_path, index=False)
            start = False

        if loop > output_num:
            break

    print(loop)
    print("#########################################")
    for i in range(output_num):
        print("##########  ", i, "  ###########")
        print("番号", len(sepalate_data[i][0])/1000)
        print("動画", len(sepalate_data[i][1])/1000)
        print("解答", len(sepalate_data[i][2])/1000)




In [ ]:
### データを分割するための関数
def separate_data3(csv_file_path, output_directory='./output/', correction_value=1):
    # 出力する数
    output_num = 120
    #サンプリング周波数 (Hz)
    sampling_frequency = 500

    # 分割されたデータを保存する
    sepalate_data = [[ [], [], [],[],[],[],[] ]]

    # トリガー
    trigger = False
    start = False
    gap = 0
    NT = False
    threshold = -100

    # 入れる場所管理用
    cnt = 0
    num = 0

    # CSVファイルをpandasのデータフレームに読み込む
    df = pd.read_csv(csv_file_path)

    # データフレームをNumPy配列に変換
    data_array = df.values
    print(int (len(data_array) / sampling_frequency))
    end_point = int (len(data_array) / sampling_frequency) -1

    for data in data_array:
        if data[4] >= threshold:
            NT = True

        if data[4] < threshold and not trigger:#開始
            trigger = True
            start = True


        elif data[4] < threshold and trigger and NT:#終了
            trigger = False
            NT = False

        if  abs(data[0] - end_point) < 1e-6: #終了で、最後はトリガーがないから、消えるの対策
            trigger = False
            NT = False

        if trigger:
            sepalate_data[cnt][num].append([data[0], data[1], data[2], data[5], data[6], data[4],data[7]])
        elif start:
            num += 1
            if num > 1:
                sepalate_data.append([ [], [], [],[],[],[],[]])
                cnt += 1
                num = 1
                print(data[0])
                dff = pd.DataFrame(sepalate_data[cnt-1][1], columns=['Time', 'C3', 'C4','Fz', 'Cz','threshold','Pz'])
                csv_output_path = f'{output_directory}outputdata{cnt-1}.csv'
                dff.to_csv(csv_output_path, index=False)

        if cnt > output_num:
            break
    print(cnt)
    print("#########################################")
    for i in range(output_num):
        print("##########  ", i, "  ###########")
        print("番号", len(sepalate_data[i][0])/1000)
        print("動画", len(sepalate_data[i][1])/1000)
        print("解答", len(sepalate_data[i][2])/1000)


In [ ]:
def process_and_save_dft(input_directory, output_directory_C3, output_directory_C4):
    # 出力ファイル名に日時を組み込む
    current_datetime = datetime.now().strftime("%Y%m%d%H%M%S")

    # 結果を格納するリスト
    result_files_C3 = []
    result_files_C4 = []

    # サンプリング周波数
    sampling_frequency = 1000

    # 入力ディレクトリ内のCSVファイルを処理
    for filename in os.listdir(input_directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_directory, filename)

            # CSVファイルをデータフレームに読み込み
            df = pd.read_csv(file_path)

            # C3列のデータを取得
            data_column_C3 = df.iloc[:, 1].values
            # C4列のデータを取得
            data_column_C4 = df.iloc[:, 2].values

            # 周波数ごとに離散フーリエ変換（DFT）を実行（C3）
            dft_results_C3 = np.fft.fft(data_column_C3)
            frequencies_C3 = np.fft.fftfreq(len(dft_results_C3), 1 / sampling_frequency)

            # 結果を新しいデータフレームに追加（C3）
            result_df_C3 = pd.DataFrame({'Frequency': frequencies_C3, 'Amplitude': np.abs(dft_results_C3)})

            # 出力ファイルのパス（C3）
            output_filename_C3 = f'{filename[:-4]}_dft_C3.csv'
            output_path_C3 = os.path.join(output_directory_C3, output_filename_C3)

            # 結果をCSVファイルに保存（C3）
            result_df_C3.to_csv(output_path_C3, index=False)
            result_files_C3.append(output_path_C3)
            print(f"DFT結果（C3）がCSVファイルに保存されました: {output_path_C3}")

            # 周波数ごとに離散フーリエ変換（DFT）を実行（C4）
            dft_results_C4 = np.fft.fft(data_column_C4)
            frequencies_C4 = np.fft.fftfreq(len(dft_results_C4), 1 / sampling_frequency)

            # 結果を新しいデータフレームに追加（C4）
            result_df_C4 = pd.DataFrame({'Frequency': frequencies_C4, 'Amplitude': np.abs(dft_results_C4)})

            # 出力ファイルのパス（C4）
            output_filename_C4 = f'{filename[:-4]}_dft_C4.csv'
            output_path_C4 = os.path.join(output_directory_C4, output_filename_C4)

            # 結果をCSVファイルに保存（C4）
            result_df_C4.to_csv(output_path_C4, index=False)
            result_files_C4.append(output_path_C4)
            print(f"DFT結果（C4）がCSVファイルに保存されました: {output_path_C4}")

    return result_files_C3, result_files_C4

In [ ]:
def process_and_save_sum_amplitude(fft_output_directory, output_csv_path, min = 8, max = 13):
    if min >= max:
      print("不正な値です。")
      sys.exit()

    # 各ファイルごとに足し合わせた結果を格納するディクショナリ
    sum_amplitude_dict = {}

    # FFToutputフォルダ内のCSVファイルを処理
    for filename in os.listdir(fft_output_directory):
        if filename.endswith(".csv"):
            # 「outputdata」に続く数字を抜き取る
            file_number = int(''.join(filter(str.isdigit, filename.split('_')[0])))

            file_path = os.path.join(fft_output_directory, filename)

            # CSVファイルをデータフレームに読み込み
            df = pd.read_csv(file_path)

            # 1列目のFrequencyが8から13Hzの部分のAmplitudeを足し合わせる
            sum_amplitude = df[(df['Frequency'] >= min) & (df['Frequency'] <= max)]['Amplitude'].sum()

            # 各ファイルの結果をディクショナリに追加
            sum_amplitude_dict[file_number] = sum_amplitude

    # キーでソートしてディクショナリをDataFrameに変換
    sorted_result = dict(sorted(sum_amplitude_dict.items()))
    result_df = pd.DataFrame(list(sorted_result.items()), columns=['Question_Number', 'Alpha_Wave'])

    # 結果をCSVファイルに保存
    result_df.to_csv(output_csv_path, index=False)

    print(f"結果がCSVファイルに保存されました: {output_csv_path}")

### メイン関数

In [ ]:

if __name__ == "__main__":
  # フォルダのパス
  output_directory = './output/'
  output_directory_C3 = './FFToutput_C3/'
  output_directory_C4 = './FFToutput_C4/'
  zip_output_directory = './output_ZIP/'
  fft_input_directory = './output/'
  output_csv_path_C3 = './alpha_wave_C3.csv'
  output_csv_path_C4 = './alpha_wave_C4.csv'

  # 読み込むCSVファイルのパス
  csv_file_path = './yoninme.csv'

  ### フォルダを作る処理

  # 出力フォルダが存在しない場合は作成
  if not os.path.exists(output_directory):
    os.makedirs(output_directory)

  # C3FFT出力フォルダが存在しない場合は作成
  if not os.path.exists(output_directory_C3):
    os.makedirs(output_directory_C3)

  # C4FFT出力フォルダが存在しない場合は作成
  if not os.path.exists(output_directory_C4):
    os.makedirs(output_directory_C4)

  # ZIP出力フォルダが存在しない場合は作成
  if not os.path.exists(zip_output_directory):
    os.makedirs(zip_output_directory)


  ### データ処理

  # データ分割
  separate_data(csv_file_path, output_directory=output_directory, correction_value=HOSEI)
  #separate_data_2(csv_file_path, output_directory=output_directory, correction_value=HOSEI)

  # FFT
  # processed_files_C3, processed_files_C4 = process_and_save_fft(fft_input_directory, output_directory_C3, output_directory_C4)

  # α波を出力
  # process_and_save_sum_amplitude(output_directory_C3, output_csv_path_C3)
  # process_and_save_sum_amplitude(output_directory_C4, output_csv_path_C4)

  # データをzip化
  compress_directory_to_zip(output_directory)
  # compress_directory_to_zip(output_directory_C3)
  # compress_directory_to_zip(output_directory_C4)


1055
1054
12.904
20.904
28.904
36.904
44.904
52.904
60.902
68.904
76.904
84.904
92.902
100.902
108.902
116.902
124.902
132.902
140.902
148.904
156.902
204.346
213.346
222.346
231.346
240.346
249.344
258.344
267.346
276.346
285.344
294.344
303.344
312.344
321.344
330.344
339.344
348.344
357.344
366.344
375.342
386.758
395.76
404.76
413.76
422.76
431.76
440.76
449.76
458.76
467.758
476.758
485.758
494.758
503.758
512.758
521.756
530.756
539.756
548.756
557.756
605.228
614.228
623.228
632.228
641.228
650.228
659.228
668.226
677.226
686.226
695.226
704.226
713.228
722.226
731.226
740.226
749.226
758.226
767.226
776.224
787.614
796.614
805.614
814.614
823.612
832.614
841.614
850.614
859.612
868.612
877.612
886.612
895.612
904.612
913.612
922.612
931.612
940.61
949.612
958.612
1054.0
1054.002
1054.004
1054.006
1054.008
1054.01
1054.012
1054.014
1054.016
1054.018
1054.02
1054.022
1054.024
1054.026
1054.028
1054.03
1054.032
1054.034
1054.036
1054.038
1054.04
1054.042
1054.044
1054.046
1054.048